# Leeds Africa Hackathon
This is a template notebook for the Leeds Africa Hackathon. It also includes some useful Python code to read CMIP6 data and CP4 data

**CMIP6:** 

Historical runs (1850-2014) are in /badc/cmip6/data/CMIP6/CMIP/*institute*/*model*/historical/*variant*/*table_id*/*variable*/*grid*/*version*/

Scenario runs are in /badc/cmip6/data/CMIP6/ScenarioMIP/*institute*/*model*/*scenario_name*/*variant*/*table_id*/*variable*/*grid*/*version*/
*scenario_name* is likely to be one of ssp119, ssp126, ssp245, ssp370 or ssp585

*scenario_name* is likely to be one of ssp119, ssp126, ssp245, ssp370 or ssp585

*variant* takes the form r0i0p0f0, e.g. r1i1p1f2, where the numbers are the indexes for:  
**r**ealization, **i**nitialization, **p**hysics, **f**orcing

*table_id* generally indicates the frequency of the data, e.g. day, 3hr, Amon

*grid* is the model grid being used, e.g. gn, where  
   * gm = global mean data  
   * gn = data reported on a model's native grid  
   * gr1 = regridded data reported on a grid other than the native grid and other than the preferred target grid  
It is likely the *grid* will be the native grid, i.e. gn

*version* normally in the form vYYYYMMDD or latest, e.g. v20200203


The following institutions have data in both historical and ScenarioMIPs:
AS-RCEC, AWI, BCC, CAMS, CAS, CCCR-IITM, CCCma, CMCC, CNRM-CERFACS, CSIRO, CSIRO-ARCCSS, E3SM-Project, EC-Earth-Consortium, FIO-QLNM, HAMMOZ-Consortium, INM, IPSL, KIOST, MIROC, MOHC, MPI-M, MRI, NASA-GISS, NCAR, NCC, NIMS-KMA, NOAA-GFDL, NUIST, THU, UA
       
**CP4:**

4km data are in /badc/impala/data/explicit-4km/*present_or_future*>/*1hr_or_mon*/*variable*/

In [ ]:
!ls /badc/cmip6/data/CMIP6/ScenarioMIP/*/*/ssp585/*/day/tas/gn/

In [ ]:
!ls /badc/cmip6/data/CMIP6/CMIP/*/*/historical

In [ ]:
!ls /badc/impala/data/explicit-4km/future/1hr/precipitation/

In [ ]:
#-----------------------------------------------------------------------------------------------------
# here is an example reading daily tas for the Met office HadGEM3-GC31-LL model for the SSP585 scenario
# from the CMIP6 archive
#-----------------------------------------------------------------------------------------------------
from read_cmip6 import *
import datetime
tas=read_cmip6('ScenarioMIP','MOHC','HadGEM3-GC31-LL','ssp585','r1i1p1f3','day','tas','gn','v20200114','20150101-20491230')
print(tas)
time_coord = tas.coord('time')
time_unit = time_coord.units
data_times = time_unit.num2date(time_coord.points)
print(data_times[0], 'to', data_times[-1])

In [ ]:
#-----------------------------------------------------------------------------------------------------
# Here is an example reading present hourly tas for the CP4 archive
# this only reads a few months as it takes a long time to read all 10 years
#-----------------------------------------------------------------------------------------------------
from read_cp4 import *
import datetime
start_year=1997
start_month=3
end_year=1997
end_month=6
tas=read_cp4('1hr','tas', start_year, start_month, end_year, end_month)
print(tas)
time_coord = tas.coord('time')
time_unit = time_coord.units
data_times = time_unit.num2date(time_coord.points)
print(data_times[0], 'to', data_times[-1])

In [ ]:
#-----------------------------------------------------------------------------------------------------
# here is an example reading all 10 future years of monthly precip for the CP4 archive
#-----------------------------------------------------------------------------------------------------
from read_cp4 import *
import datetime
start_year=2095
start_month=3
end_year=2105
end_month=2
pr=read_cp4('mon','pr', start_year, start_month, end_year, end_month)
print(pr)
time_coord = pr.coord('time')
time_unit = time_coord.units
data_times = time_unit.num2date(time_coord.points)
print(data_times[0], 'to', data_times[-1])